In [ ]:
%pip install numpy scikit-learn

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

# Consigna 1

Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos. Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

In [4]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In [5]:
# instanciamos un vectorizador, por el momento usamos con parametros por default
tfidfvect = TfidfVectorizer()

In [6]:
X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [7]:
# Obtengo los datos y labels de test para la consigna siguiente
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target

In [8]:
# obtenemos 5 numeros aleatorios con el mismo valor de seed (para repetibilidad) entre 1 y 11314 (cantidad de documentos)
import random
random.seed(42)
rand_docs_idx = [random.randint(1,X_train.shape[0]) for _ in range(5)]
print(rand_docs_idx)

[10477, 1825, 410, 4507, 4013]


In [9]:
# Obtenemos la similitud coseno de los 5 documentos obtenidos en el paso anterior
for rand_doc_idx in rand_docs_idx:
    print("=" * 100)
    print(f"Documento ID {rand_doc_idx} - Categoria: {newsgroups_train.target_names[y_train[rand_doc_idx]]} - Resultados similiud:")
    print("=" * 100)
    #
    print(f"Contenido:")
    print(f"{newsgroups_train.data[rand_doc_idx][:1000]}")
    print("-" * 100)
    print(f"\n")
    print(f"Documentos similares: \n")
    cossim = cosine_similarity(X_train[rand_doc_idx], X_train)[0]
    # los 5 documentos más similares
    sim_val = np.sort(cossim)[::-1][1:6]
    sim_doc = np.argsort(cossim)[::-1][1:6]

    for idx,val in enumerate(sim_val):
        #
        print(f"{idx+1}- Documento ID: {sim_doc[idx]} - Categoria: {newsgroups_train.target_names[y_train[sim_doc[idx]]]} - Similitud: {val}")
        print(f"\n")
        print(f"Contenido:")
        print(f"{newsgroups_train.data[sim_doc[idx]][:1000]}")
        print("*" *100)
    print(f"\n")
    
    




Documento ID 10477 - Categoria: alt.atheism - Resultados similiud:
Contenido:

Then Mr Mozumder is incorrect when he says that when committing
bad acts, people temporarily become atheists?
----------------------------------------------------------------------------------------------------


Documentos similares: 

1- Documento ID: 4426 - Categoria: alt.atheism - Similitud: 0.22819690349790384


Contenido:

And doubtless, when an atheist does an act of charity
they temporarily become a Baptist.
****************************************************************************************************
2- Documento ID: 10108 - Categoria: alt.atheism - Similitud: 0.2011158912352038


Contenido:

Of course B.M. is not incorrect.  He is defending Islam.  When defending
Islam against infidels you can say anything and no one will dare criticize
you.  But when an atheist uses the same argument he is using "petty sarcasm".  So
B.M. can have his "temporary atheists" whenever he needs them and all the
"t

## Analisis de resultados

### Documento 1:

El documento 1 (ID = 10477), de categoria "Ateísmo" se relaciona con los 5 documentos mencionados, ya que su contenido está relaciondo con los "malos actos" o "malas acciones" y el ateísmo, mientras que los primeros 3, no solamente coinciden con la categoría sino que tambien mencionan palabras como "atheist" o "temporary atheist"; y los ultimos dos, por mas que pertenezcan a las categorías "Religión Cristiana" y "Políticas (Misceláneo)", su contenido está fuertemente relacionado con las creencias y el ateísmo.

### Documento 2:

El documento 2 (ID = 1825), de categoría "Autos", se caracteriza por ser muy corto, de solamente 8 palabras, generando que la realación que se obtenga con los demás documentos esté fuertemente relacionado por la mención explicita de algunas de esas palabras. Para el primer caso, el contenido es solamente la palabra "Huh?" (y por lo tanto concentra el 100% de la información) la cual se menciona en el mensaje original; para el segundo caso tambien se menciona la palabra "huh" en dos ocasiones pero dentro de un mensaje un poco más extenso; para el tercer caso se menciona la palabra "Beamers", la cual se encuentra presente en el mensaje original; para el cuarto caso se menciona el acrónimo "IBM"; y para el quinto caso se menciona la palabra "Beamers". Generando así unas discrepancia entre la categoría de referencia y los documentos 1, 2 y 4, mientras que coincide con los documentos 3 y 5, ya que el termino "Beamers" corresponde con el auto marca "BWM".

### Documento 3:

El documento 3 (ID = 410), de categoría "Política - Armas", habla sobre los proyectiles y su letalidad, sobre proyectiles de goma para situaciones de disturbios ("Riots"), la legítima defensa con armas de fuego y características sobre armas "poco letales". El primer documento, por más que pertenezca a la categoria "Ciencia", trata sobre el uso lasers como armas; el segundo, tercer y cuarto documento, pertenecen a la misma categoría y hablan directamente sobre armas; finalmente el quinto documento, por más que pertenece a la categoría "Ateísmo", el mismo presenta FAQs que estan relacionadas con la muerte, la política, etc.

### Documento 4:

El documento 4 (ID = 4507), de categoría "Política - Medio Orienete", habla sobre la conflictividad que existe entre el pueblo Turco y el pueblo Griego. El primero documento hace una referencia explícita de la primer frase del documento a analizar, inferiendo que se trata de una respuesta al mismo; el segundo, tercer, cuarto y quinto documento también mencionan las relaciones que existen entre Grecia y Turquía, y la conflictividad derivada en Chipre. Para este caso, los cinco documentos más similares pertenecen a la misma categoría, "Política - Medio Oriente".

### Documento 5:

El documento 5 (ID = 4013), de categoria "Ciencia - Espacio", habla sobre la necesidad de desarrollar sistemas de despegue de bajo presupuesto. El primer, segundo y cuarto documento, pertenecientes a la misma categoría, estan directamente relacionados con la temática propuesta en el documento de referencia; el tercer documento, por más que no hace alusión explícita a dicho sistema, si es un texto acerca del espacio exterior, lo cual genera una gran similitud con el texto de referencia; el quinto documento, pertenece a otra categoría, "Política - Medio Oriente", y no se encuentra relacionado con la temática decrita anteriormente, por lo que se estima que la similitud esta dada por las otras palabras que componen al documento de referencia.


# Consigna 2
Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

Inicialmente hacemos una prueba con un vectorizador seteado por default y los modelos MultinomialNB y ComplementNB, también por default.

In [10]:
tfidfvect = TfidfVectorizer()

In [11]:
X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target

In [12]:
mult_clf = MultinomialNB()
mult_clf.fit(X_train, y_train)


MultinomialNB()

In [13]:
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  mult_clf.predict(X_test)
f1 = f1_score(y_test, y_pred, average='macro')
print(f"MultinomialNB: f1_score = {f1}")

MultinomialNB: f1_score = 0.5854345727938506


In [14]:
comp_clf = ComplementNB()
comp_clf.fit(X_train,y_train)

ComplementNB()

In [15]:
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  comp_clf.predict(X_test)
f1 = f1_score(y_test, y_pred, average='macro')
print(f"MultinomialNB: f1_score = {f1}")

MultinomialNB: f1_score = 0.692953349950875


Se observa una mejora del 18% utilizando el clasificador 'ComplementNB' comparado con el clasificador 'MultinomialNB'.

Para variar los modelos vamos a utilizar distintos valores del parámetro 'alpha', parámetro de smoothing; mientras que para el vectorizador vamos utilizar una combinación lineal de los siguientes parámetros:  'norm', que determina que tipo de norma se utiliza (l1 o l2); 'max_df', parámetro que ignora las palabras que tienen una frecuencia mayor a la del valor establecido por el parámetro; y 'min_df', parámetro que ignora las palabras que tienen una frecuencia menor a la del valor establecido por el parámetro.

In [16]:
# Parametro de smothing de los modelos
alpha_list = [0.2, 1, 5 , 7, 10]

# Parametros del vectorizador
norm_list = ['l1','l2']
max_df_list = [0.6, 0.7, 0.8, 0.9, 1.0]
min_df_list = [0.0, 0.1, 0.2, 0.3, 0.4]

results = []

for norm in norm_list:
    for max_df in max_df_list:
        for min_df in min_df_list:

            tfidfvect = TfidfVectorizer(
                norm    =   norm,
                max_df  =   max_df,
                min_df  =   min_df 
            )
            X_train = tfidfvect.fit_transform(newsgroups_train.data)
            X_test = tfidfvect.transform(newsgroups_test.data)
            
            for alpha in alpha_list:
                mult_clf = MultinomialNB(alpha=alpha)
                mult_clf.fit(X_train, y_train)
                y_pred =  mult_clf.predict(X_test)
                f1_val = f1_score(y_test, y_pred, average='macro')
                result = {
                    'classifier':   'MultinomialNB',
                    'norm':         norm,
                    'max_df':       max_df,
                    'min_df':       min_df,
                    'alpha':        alpha,
                    'f1_score':     f1_val,
                }
                results.append(result)
                comp_clf = ComplementNB(alpha=alpha)
                comp_clf.fit(X_train, y_train)
                y_pred =  comp_clf.predict(X_test)
                f1_val = f1_score(y_test, y_pred, average='macro')
                result = {
                    'classifier':   'ComplementNB',
                    'norm':         norm,
                    'max_df':       max_df,
                    'min_df':       min_df,
                    'alpha':        alpha,
                    'f1_score':     f1_val,
                }
                results.append(result)




In [17]:
# Ordenar los resultados segun el f1-score
sorted_results = sorted(results, key=lambda x: x['f1_score'], reverse=True)
for i, result in enumerate(sorted_results[:3], start=1):
    print(f"Result {i}:")
    print(f"  Classifier: {result['classifier']}")
    print(f"  Normalization: {result['norm']}")
    print(f"  Max DF: {result['max_df']}")
    print(f"  Min DF: {result['min_df']}")
    print(f"  Alpha: {result['alpha']}")
    print(f"  F1 Score: {result['f1_score']:.4f}")
    print('-' * 40)

Result 1:
  Classifier: ComplementNB
  Normalization: l2
  Max DF: 0.7
  Min DF: 0.0
  Alpha: 0.2
  F1 Score: 0.7002
----------------------------------------
Result 2:
  Classifier: ComplementNB
  Normalization: l2
  Max DF: 0.8
  Min DF: 0.0
  Alpha: 0.2
  F1 Score: 0.7001
----------------------------------------
Result 3:
  Classifier: ComplementNB
  Normalization: l2
  Max DF: 0.6
  Min DF: 0.0
  Alpha: 0.2
  F1 Score: 0.7000
----------------------------------------


## Análisis de Resultados

Se observa que los casos con mayor 'f1-score' son aquellos en donde se utiliza el clasificador 'ComplementNB' con un alpha igual a '0.2' y un 'min_df' igual 0. Si comparamos los resultados de los mejores 3 casos, observamos que el cambio del max_df no genera una mejora significativa entre ellos ya que la variación es menor al 1%. Es más, comparando el resultado entre el mejor caso y el resultado con el vectorizador y el clasificador 'ComplementNB' seteados por defecto, difieren únicamente en un 2%.


# Consigna 3

Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras. Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

Utilizando el mejor vectorizador, obtenido en la consigna anterior

In [ ]:
import re

# Utilizando un custom preprocessor podemos filtrar muchas palabras sin sentido dentro del vocabulario como '56kb','_______', 'a__b', 'a_', '_b', etc.
def custom_preprocessor(text):
   # Convert text to lowercase
    text = text.lower()
    text = re.sub(r'\b\w*\d\w*\b', '', text)
    text = re.sub(r'\b_+\b', '', text)
    text = re.sub(r'\b\w+_+\w+\b', '', text)
    text = re.sub(r'\b_+\w+_+\b', '', text)
    text = re.sub(r'\b\w+_+\b', '', text)
    text = re.sub(r'\b_+\w+\b', '', text)

    return text

In [25]:
norm = 'l2'
max_df = 0.7
min_df = 0.0

tfidfvect = TfidfVectorizer(
                norm    =   norm,
                max_df  =   max_df,
                min_df  =   min_df,
                stop_words='english',
                preprocessor=custom_preprocessor
            )

X_train = tfidfvect.fit_transform(newsgroups_train.data)


In [56]:
# Obtengo 5 palabras 
vocabulary = tfidfvect.get_feature_names_out()

words_idx = [20100,30201,41219,50110,56010]

print(vocabulary[words_idx[0]])
print(vocabulary[words_idx[1]])
print(vocabulary[words_idx[2]])
print(vocabulary[words_idx[3]])
print(vocabulary[words_idx[4]])

words = [vocabulary[words_idx[0]],vocabulary[words_idx[1]],vocabulary[words_idx[2]],vocabulary[words_idx[3]],vocabulary[words_idx[4]]]


feedbacks
jones
olive
roading
structures


In [57]:
# Obtenemos la similitud coseno de las 5 palabras obtenidas en el paso anterior
for word_idx,word in enumerate(words):
    print("=" * 100)
    print(f"Palabra '{word}'")
    print("=" * 100)
    print(f"Palabras similares: \n")
    cossim = cosine_similarity(X_train.T[words_idx[word_idx]], X_train.T)[0]
    # los 5 documentos más similares
    sim_val = np.sort(cossim)[::-1][1:6]
    sim_words = np.argsort(cossim)[::-1][1:6]

    for idx,val in enumerate(sim_val):
        print(f"{idx+1}- Palabra: {vocabulary[sim_words[idx]]} - Similitud: {val}")
    print(f"\n")

Palabra 'feedbacks'
Palabras similares: 

1- Palabra: actuarial - Similitud: 0.9924403142925359
2- Palabra: insurers - Similitud: 0.8362952994167565
3- Palabra: premium - Similitud: 0.7669862582328717
4- Palabra: preventative - Similitud: 0.7285880032912484
5- Palabra: illustrative - Similitud: 0.7090730915787998


Palabra 'jones'
Palabras similares: 

1- Palabra: plao - Similitud: 0.4003329597040183
2- Palabra: thad - Similitud: 0.3917785254707796
3- Palabra: serenade - Similitud: 0.3917785254707796
4- Palabra: percy - Similitud: 0.3917785254707796
5- Palabra: penthouse - Similitud: 0.3917785254707796


Palabra 'olive'
Palabras similares: 

1- Palabra: hitched - Similitud: 0.97539198357275
2- Palabra: rededicate - Similitud: 0.97539198357275
3- Palabra: sergeant - Similitud: 0.97539198357275
4- Palabra: koon - Similitud: 0.97539198357275
5- Palabra: revitalization - Similitud: 0.97539198357275


Palabra 'roading'
Palabras similares: 

1- Palabra: winch - Similitud: 0.7963873172120454


## Analisis de resultados

En este caso la similitud esta basada en la repetición de las palabras dentro de los documentos, lo que genera que las palabras similares no sean similares en significado o de la misma familia. Sin embargo, se observan algunos casos peculiares como la palabra "jones", la cual se relaciona con la palabra "percy", lo cual puede hacer alusión al bajista "Percy Jones", el cual, al ser músico esta relacionado con la palabra "serenade"